In [293]:
from random import *
class Maze:
    """
    Classe Labyrinthe
    Représentation sous forme de graphe non-orienté
    dont chaque sommet est une cellule (un tuple (l,c))
    et dont la structure est représentée par un dictionnaire
      - clés : sommets
      - valeurs : ensemble des sommets voisins accessibles
    """
    def __init__(self, height, width,empty : bool):
        """
        Constructeur d'un labyrinthe de height cellules de haut 
        et de width cellules de large 
        Les voisinages sont initialisés à des ensembles vides puis si empty est True les voisinages
        remplis avec toutes les cellules contigües dans la grille
        """
        
        self.height    = height # hauteur (ligne)
        self.width     = width  # largeur (col)
        self.neighbors = {(i,j): set() for i in range(height) for j in range (width)}
        if empty:
            for c in self.neighbors.keys():
                lstDef = set()
                lstTmp = [(c[0] + 1, c[1]), (c[0] - 1, c[1]), (c[0], c[1] + 1), (c[0], c[1] - 1)]
                for elt in lstTmp:
                    if elt[0] >= 0 and elt[0] < self.height and elt[1] >= 0 and elt[1] < self.width:
                        lstDef.add(elt)
                self.neighbors[c] = lstDef
                       
    def info(self):
        """
        **NE PAS MODIFIER CETTE MÉTHODE**
        Affichage des attributs d'un objet 'Maze' (fonction utile pour deboguer)
        Retour:
            chaîne (string): description textuelle des attributs de l'objet
        """
        txt = "**Informations sur le labyrinthe**\n"
        txt += f"- Dimensions de la grille : {self.height} x {self.width}\n"
        txt += "- Voisinages :\n"
        txt += str(self.neighbors)+"\n"
        valid = True
        for c1 in {(i, j) for i in range(self.height) for j in range(self.width)}:
            for c2 in self.neighbors[c1]:
                if c1 not in self.neighbors[c2]:
                    valid = False
                    break
            else:
                continue
            break
        txt += "- Structure cohérente\n" if valid else f"- Structure incohérente : {c1} X {c2}\n"
        return txt

    def __str__(self):
        """
        **NE PAS MODIFIER CETTE MÉTHODE**
        Représentation textuelle d'un objet Maze (en utilisant des caractères ascii)
        Retour:
             chaîne (str) : chaîne de caractères représentant le labyrinthe
        """
        txt = ""
        # Première ligne
        txt += "┏"
        for j in range(self.width-1):
            txt += "━━━┳"
        txt += "━━━┓\n"
        txt += "┃"
        for j in range(self.width-1):
            txt += "   ┃" if (0,j+1) not in self.neighbors[(0,j)] else "    "
        txt += "   ┃\n"
        # Lignes normales
        for i in range(self.height-1):
            txt += "┣"
            for j in range(self.width-1):
                txt += "━━━╋" if (i+1,j) not in self.neighbors[(i,j)] else "   ╋"
            txt += "━━━┫\n" if (i+1,self.width-1) not in self.neighbors[(i,self.width-1)] else "   ┫\n"
            txt += "┃"
            for j in range(self.width):
                txt += "   ┃" if (i+1,j+1) not in self.neighbors[(i+1,j)] else "    "
            txt += "\n"
        # Bas du tableau
        txt += "┗"
        for i in range(self.width-1):
            txt += "━━━┻"
        txt += "━━━┛\n"

        return txt
    
    def add_wall(self, c1, c2):
        # Facultatif : on teste si les sommets sont bien dans le labyrinthe
        assert  0 <= c1[0] < self.height and \
                0 <= c1[1] < self.width and \
                0 <= c2[0] < self.height and \
                0 <= c2[1] < self.width, \
            f"Erreur lors de l'ajout d'un mur entre {c1} et {c2} : les coordonnées de sont pas compatibles avec les dimensions du labyrinthe"
        # Ajout du mur
        if c2 in self.neighbors[c1]:      # Si c2 est dans les voisines de c1
            self.neighbors[c1].remove(c2) # on le retire
        if c1 in self.neighbors[c2]:      # Si c3 est dans les voisines de c2
            self.neighbors[c2].remove(c1) # on le retire
            
    def remove_wall(self, c1, c2):
        """
        Supprime le mur (si il y en a un) entre deux cellules (c1 et c2)
        
        Params:
            - c1 : cellule
            - c2 : cellule
        """
        assert  0 <= c1[0] < self.height and \
                0 <= c1[1] < self.width and \
                0 <= c2[0] < self.height and \
                0 <= c2[1] < self.width, \
            f"Erreur lors de la suppression d'un mur entre {c1} et {c2} : les coordonnées de sont pas compatibles avec les dimensions du labyrinthe"
        
        # Suppression du mur
        if not c2 in self.neighbors[c1]:  # Si c2 n'est pas dans les voisines de c1
            self.neighbors[c1].add(c2)    # on l'ajoute
        if not c1 in self.neighbors[c2]:  # Si c3 n'est pas dans les voisines de c2
            self.neighbors[c2].add(c1)    # on l'ajoute
    
    def get_walls(self):
        """
        Retourne la liste de tous les murs sous la forme d’une liste de tuple de cellules
        
        Retour:
            - Liste de liste de deux cellules
        """
        
        L = []
        
        for c1 in {(i, j) for i in range(self.height) for j in range(self.width)}:
            
            # cherche les murs situé à droite de c1
            if (c1[0], c1[1]+1) not in self.neighbors[c1] and c1[1] < self.width-1:
                L.append([c1, (c1[0], c1[1]+1)])
            
            # cherche les murs situé en dessous de c1
            if (c1[0]+1, c1[1]) not in self.neighbors[c1] and c1[0] < self.height-1:
                L.append([c1, (c1[0]+1, c1[1])])
                
        return L
    
    def fill(self):
        """
        Place le plus de murs possible dans le labyrinthe
        """
        for c1 in {(i, j) for i in range(self.height) for j in range(self.width)}:
            if (c1[0], c1[1]+1) in self.neighbors[c1] and c1[1] < self.width-1:
                self.add_wall(c1, (c1[0], c1[1]+1))
            
            if (c1[0]+1, c1[1]) in self.neighbors[c1] and c1[0] < self.height-1:
                self.add_wall(c1, (c1[0]+1, c1[1]))

    def empty(self):
        """
        Retire tous les murs du labyrinthe
        """
        for c1 in {(i, j) for i in range(self.height) for j in range(self.width)}:
            if (c1[0], c1[1]+1) not in self.neighbors[c1] and c1[1] < self.width-1:
                self.remove_wall(c1, (c1[0], c1[1]+1))
            
            if (c1[0]+1, c1[1]) not in self.neighbors[c1] and c1[0] < self.height-1:
                self.remove_wall(c1, (c1[0]+1, c1[1]))
                
    def get_contiguous_cells(self, c):
        """
        retourne la liste des cellules contigües à c dans la grille sans s’occuper des éventuels murs
        """
        lstCont = []
        lstTmp = [(c[0] + 1, c[1]), (c[0] - 1, c[1]), (c[0], c[1] + 1), (c[0], c[1] - 1)]
        for elt in lstTmp:
            if elt[0] >= 0 and elt[0] < self.height and elt[1] >= 0 and elt[1] < self.width:
                lstCont.append(elt)
            
        return lstCont
    
    def get_reachable_cells(self, c):
        """
        retourn la liste de cellules voisine et sans mur entre elle et c dans la grille
        """
        lstTemp = self.get_contiguous_cells(c)
        lstRep = []
        for elt in lstTemp:
            if elt in self.neighbors[c]:
                lstRep.append(elt)
        return lstRep
    
    @classmethod 
    def gen_btree(cls, h, w):
        """
        génére un labyrinthe sous la d'un arbre binaire 
        """
        laby = Maze(h, w, False)
        for cel in laby.neighbors:
            lstVoisins = laby.get_contiguous_cells(cel)
            lstTemp = []
            for elt in lstVoisins:
                # on regarde ci il y a des mure E et S
                if (elt[0] == cel[0] or elt[0] == cel[0]+1) and (elt[1] == cel[1] or elt[1] == cel[1]+1):
                    lstTemp.append(elt)
            if len(lstTemp) == 1:
                laby.remove_wall(lstTemp[0], cel)
            elif lstTemp:
                laby.remove_wall(choice(lstTemp), cel)
        return laby
    
    @classmethod
    def gen_sidewinder(cls, h, w):
        """
        génére un labyrinthe ligne par ligne
        en supriment tout les mure sur l'axe Est-West de la dernière ligne
        """
        laby = Maze(h, w, False)
        for i in range(h - 1):
            seq = []
            for j in range(w - 1):
                seq.append((i,j))
                # 0 pile , 1 face
                tirage = randint(0,1)
                if not tirage: #ci c'est pile
                    laby.remove_wall((i,j),(i,j + 1))
                elif tirage : # ci c'est face
                    cel = choice(seq)
                    laby.remove_wall(cel,(cel[0] + 1, cel[1]))
                    seq = []
            seq.append((i,j + 1))
            cel = choice(seq)
            laby.remove_wall(cel,(cel[0] + 1, cel[1]))
        i = h - 1
        for j in range(w - 1):
            laby.remove_wall((i, j),(i, j + 1))
        return laby
    
    @classmethod
    def gen_exploration(cls, h, w):
        """
        génére un labyrinthe en explorant aléatoirement ce dernier
        et en faisent tmber les murs ou fur et a mesure
        """
        laby = Maze(h, w, False)
        lign = randint(0, h - 1)
        col = randint(0, w - 1)
        cel = (lign, col)
        marque = [cel]
        pile = [cel]
        
        while pile: #tant que la plie n'est pas vide
            cel = pile.pop()
            lstVois = laby.get_contiguous_cells(cel)
            pasMarquer = False
            curs = 0
            while not pasMarquer and curs < len(lstVois):
                if lstVois[curs] not in marque:
                    pasMarquer = True
                curs +=1
            if pasMarquer:
                pile.append(cel)
                choix = True
                while choix: # tant que on fait un choix
                    choisi = choice(lstVois)
                    if choisi not in marque:
                        choix = False
                laby.remove_wall(choisi, cel)
                marque.append(choisi)
                pile.append(choisi)
        return laby
        
    @classmethod
    def gen_fusion(cls, h, w):
        """
        génère un labyrinthe, à h lignes et w colonnes parfait en fusionnent les chemin 
        """
        # on remplit le labyrinthe avec tous les murs possibles
        laby = Maze(h, w, False)
        
        # on labélise les cellules de 1 à n
        liste_cellules = list(laby.neighbors.keys())
        cellules_label = {}
        for cell in liste_cellules:
            cellules_label[cell] = liste_cellules.index(cell)+1
        
        # on extrait la liste de tous les murs et on les « mélange » (on les permute aléatoirement)
        walls = laby.get_walls()
        shuffle(walls)
        
        for wall in walls:
            if cellules_label[wall[0]] != cellules_label[wall[1]]:
                laby.remove_wall(wall[0], wall[1])
                
                nouveau_label = cellules_label[wall[0]]
                ancien_label = cellules_label[wall[1]]               
                for cell, label in cellules_label.items():
                    if label == ancien_label:
                        cellules_label[cell] = nouveau_label
                
        return laby
    
            
    @classmethod
    def gen_wilson(cls, h, w):
        """
        génère un labyrinthe, à h lignes et w colonnes en parcourent aléatoirement le labyrinthe et en cassent les murs 
        à la manière d’un parcours en profondeur
        """
        laby = Maze(h, w, False)
        cels = [(i,j) for i in range(h) for j in range (w)]
        celMarque = choice(cels)
        cels.remove(celMarque)
        marque = [celMarque]
        while cels:
            dep = choice(cels)
            cels.remove(dep)
            att = False
            lstChem = [dep]
            while not att:
                step = choice(laby.get_contiguous_cells(lstChem[-1]))
                if step in marque:
                    lstChem.append(step)
                    marque += lstChem
                    for i in range(len(lstChem) - 1):
                        laby.remove_wall(lstChem[i],lstChem[i + 1])
                    att = True
                    
                elif step in lstChem:
                    cels += lstChem
                    att = True
                else:
                    lstChem.append(step)
                    cels.remove(step)
            lstChem = []
        return laby
    
    def overlay(self, content=None):
        """
        Rendu en mode texte, sur la sortie standard, \
        d'un labyrinthe avec du contenu dans les cellules
        Argument:
            content (dict) : dictionnaire tq content[cell] contient le caractère à afficher au milieu de la cellule
        Retour:
            string
        """
        if content is None:
            content = {(i,j):' ' for i in range(self.height) for j in range(self.width)}
        else:
            # Python >=3.9
            #content = content | {(i, j): ' ' for i in range(
            #    self.height) for j in range(self.width) if (i,j) not in content}
            # Python <3.9
            new_content = {(i, j): ' ' for i in range(self.height) for j in range(self.width) if (i,j) not in content}
            content = {**content, **new_content}
        txt = r""
        # Première ligne
        txt += "┏"
        for j in range(self.width-1):
            txt += "━━━┳"
        txt += "━━━┓\n"
        txt += "┃"
        for j in range(self.width-1):
            txt += " "+content[(0,j)]+" ┃" if (0,j+1) not in self.neighbors[(0,j)] else " "+content[(0,j)]+"  "
        txt += " "+content[(0,self.width-1)]+" ┃\n"
        # Lignes normales
        for i in range(self.height-1):
            txt += "┣"
            for j in range(self.width-1):
                txt += "━━━╋" if (i+1,j) not in self.neighbors[(i,j)] else "   ╋"
            txt += "━━━┫\n" if (i+1,self.width-1) not in self.neighbors[(i,self.width-1)] else "   ┫\n"
            txt += "┃"
            for j in range(self.width):
                txt += " "+content[(i+1,j)]+" ┃" if (i+1,j+1) not in self.neighbors[(i+1,j)] else " "+content[(i+1,j)]+"  "
            txt += "\n"
        # Bas du tableau
        txt += "┗"
        for i in range(self.width-1):
            txt += "━━━┻"
        txt += "━━━┛\n"
        return txt
    
    def solve_dfs(self, D, A):
        """
        Algorithme de résolution de labyrinthe par parcours
        
        Params:
            D : cellule de départ
            A : cellule d'arrivée
        """
        attente = [D]
        predecesseur = {D: D}
        cellules = list(laby.neighbors.keys())
        non_marque = list(laby.neighbors.keys())
        non_marque.remove(D)
        
        for _ in cellules:
            cell = attente.pop()
            if cell == A:
                c = A
                chemin = []
                while c != D:
                    chemin.append(c)
                    c = predecesseur[c]
                return chemin
            
            else:
                for r_cell in self.get_reachable_cells(cell):
                    if r_cell in non_marque:
                        non_marque.remove(r_cell)
                        attente.append(r_cell)
                        predecesseur[r_cell] = cell
    
    def solve_bfs(self, dep, ar):
        """
        Algorithme de résolution de labyrinthe par parcours en largeur
        
        Params:
            dep : cellule de départ
            ar : cellule d'arrivée
        """
        file = [dep]
        predecesseur = {dep: dep}
        cellules = list(laby.neighbors.keys())
        non_marque = list(laby.neighbors.keys())
        non_marque.remove(dep)
        
        for _ in cellules:
            cell = file.pop(0)
            if cell == ar:
                c = ar
                chemin = []
                while c != dep:
                    chemin.append(c)
                    c = predecesseur[c]
                return chemin
            
            else:
                for r_cell in self.get_reachable_cells(cell):
                    if r_cell in non_marque:
                        non_marque.remove(r_cell)
                        file.append(r_cell)
                        predecesseur[r_cell] = cell

#fonction non demander
    def getProchCelVer(self, dire, pose):
        """
        fonction qui renvois la prochaine cel dans dans la direction dire demander par raport a la cel pose donner
        """
        if dire == "N":
            nextPose = (pose[0] - 1, pose[1])
        elif dire == "S":
            nextPose = (pose[0] + 1, pose[1])
        elif dire == "E":
            nextPose = (pose[0], pose[1] + 1)
        elif dire == "W":
            nextPose = (pose[0], pose[1] - 1)
        return nextPose
    
#fonction non demander
    def getCelDroit(self, dire, pose):
        """
        renvoi un tuple composer de la cellule a la droit de notre position qu'il y est un mur ou non en fonction de 
        notre direction ainsi que la nouvel direction ou None ci il n'y a pas de cellule a droit comme le long d'un bord 
        du laby
        """
        lstVois = self.get_contiguous_cells(pose)
        if dire == "N": # droit de N est E
            celDroit = (pose[0], pose[1] + 1)
            dire = "E"
            
        elif dire == "S": #droit de S est W
            celDroit = (pose[0], pose[1] - 1)
            dire = "W"
            
        elif dire == "E": # droit de E est S
            celDroit = (pose[0] + 1, pose[1])
            dire = "S"
            
        elif dire == "W": # droit de w est N
            celDroit = (pose[0] - 1, pose[1])
            dire = "N"
            
        if celDroit not in lstVois:
            celDroit = None
            dire = None
        return (celDroit, dire)
    
#fonction non demander
    def getCelGauche(self, dire, pose):
        """
        renvoi un tuple composer de la cellule a la gauche de notre position qu'il y est un mur ou non en fonction de 
        notre direction ainsi que la nouvel direction ou None ci il n'y a pas de cellule a gauche comme le long d'un bord 
        du laby
        """
        lstVois = self.get_contiguous_cells(pose)
        if dire == "N": # gauche de N est W
            celGauche = (pose[0], pose[1] - 1)
            dire = "W"
            
        elif dire == "S": # gauche de S est E
            celGauche = (pose[0], pose[1] + 1)
            dire = "E"
            
        elif dire == "E": # gauche de E est N
            celGauche = (pose[0] - 1, pose[1])
            dire = "N"
            
        elif dire == "W": # gauche de W est S
            celGauche = (pose[0] + 1, pose[1])
            dire = "S"
            
        if celGauche not in lstVois:
            celGauche = None
            dire = None
        return (celGauche, dire)
        
    def solve_rhr(self, start, stop):
        """
        Algorithme de résolution de labyrinthe dit de la main droite 
        """
        # dire est la direction ou la personne avance dir possible : N, S, E, W
        dire = "S"
        pose = start
        chemin = [start]
        while pose != stop:
            lstVois = self.get_reachable_cells(pose)
            droit = self.getCelDroit(dire, pose)
            if droit[0] not in lstVois:
                if self.getProchCelVer(dire, pose) not in lstVois:
                    gauche = self.getCelGauche(dire, pose)
                    if gauche[0] in lstVois:
                        pose = gauche[0]
                        dire = gauche[1]
                        chemin.append(pose)
                    else:
                        if dire == "N":
                            dire = "S"
                        elif dire == "S":
                            dire = "N"
                        elif dire == "E":
                            dire = "W"
                        elif dire == "W":
                            dire = "E"
                    
                else:
                    pose = self.getProchCelVer(dire, pose)
                    chemin.append(pose)
            else:
                pose = droit[0]
                dire = droit[1]
                chemin.append(pose)
        return chemin
                    
                    
    def distance_geo(self, c1, c2):
        return len(self.solve_bfs(c1, c2))
        
    def distance_man(self, c1, c2):
        return abs(c1[0] - c2[0]) + abs(c1[1] - c2[1])
            

    def difficiculty(self, c1, c2):
        return self.distance_geo(c1, c2) / self.distance_man(c1, c2)
                

In [305]:
laby = Maze(5, 5, empty=True)
#print(laby.info())
#print(laby)
laby = Maze.gen_fusion(10, 10)
laby.solve_dfs((0, 0), (9, 9))
for i in range(1000):
    laby = Maze.gen_fusion(10, 10)
    print(round(laby.difficiculty((0, 0), (9, 9)), 2))

1.11
1.11
1.56
1.22
1.33
1.22
1.33
1.33
1.22
1.22
1.33
1.22
1.44
1.11
1.33
1.22
1.11
1.22
1.0
1.0
1.0
1.22
1.89
1.44
1.11
1.11
1.44
1.56
1.0
1.56
1.0
1.11
1.11
1.56
1.11
1.44
1.22
1.11
1.44
1.33
1.33
1.78
1.0
1.11
1.33
1.22
1.11
1.22
1.22
1.0
1.22
1.33
1.22
1.67
1.33
1.44
1.22
1.11
1.33
1.67
1.22
1.33
1.33
1.78
1.11
1.33
1.11
1.33
1.11
1.11
1.89
1.11
1.44
1.22
1.11
1.0
1.22
1.56
1.89
1.0
2.22
1.33
1.33
1.22
1.11
1.22
1.33
1.89
1.33
1.56
1.56
1.11
1.22
1.11
1.33
1.0
1.44
1.0
1.0
1.0
1.33
1.0
1.44
1.11
1.33
1.22
1.11
1.0
1.33
1.11
1.22
1.22
1.11
1.11
1.56
1.89
1.0
1.11
1.0
1.67
2.11
1.33
1.11
1.44
1.22
1.33
1.11
1.0
1.22
1.44
1.22
1.33
1.67
1.22
1.33
1.22
1.22
1.44
1.33
1.11
1.33
1.0
1.33
1.0
1.22
1.44
1.0
1.56
1.22
1.22
1.22
1.0
1.44
1.11
1.44
1.22
1.11
1.11
1.22
1.67
1.0
1.44
1.44
1.11
1.33
1.22
1.44
1.22
1.11
1.22
1.56
1.0
1.0
1.11
1.56
1.11
1.22
1.22
1.44
2.33
1.11
1.11
1.33
1.0
1.33
1.11
1.11
1.11
1.22
2.0
1.0
1.44
1.11
1.44
1.11
1.11
1.56
1.11
1.33
1.33
1.22
1.0
1.56
1.0
1.33
1.56


In [3]:
# 4 Manipulation de labyrinthes

laby.add_wall((1, 1), (1, 2))
laby.add_wall((2, 1), (1, 1))
#print(laby)
laby.remove_wall((1, 1), (1, 2))
#print(laby)

In [4]:
laby.get_walls()

[[(1, 1), (2, 1)]]

In [5]:
laby.fill()
#print(laby)

laby.empty()
#print(laby)

In [6]:
laby.empty()
laby.add_wall((0, 0), (0, 1))
laby.add_wall((0, 1), (1, 1))
#print(laby)

In [7]:
print(laby.get_contiguous_cells((0,1)))
print(laby.get_reachable_cells((0,1)))

[(1, 1), (0, 2), (0, 0)]
[(0, 2)]


In [8]:
laby = Maze.gen_btree(4, 4)
#print(laby)

In [9]:
laby = Maze.gen_sidewinder(4, 4)
#print(laby)

In [10]:
laby = Maze.gen_exploration(15,15)
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃               ┃               ┃       ┃               ┃
┣   ╋   ╋   ╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋   ╋   ╋   ╋   ╋━━━╋━━━┫
┃       ┃               ┃   ┃           ┃   ┃   ┃   ┃       ┃
┣   ╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋   ╋   ╋   ╋   ┫
┃           ┃           ┃   ┃               ┃   ┃       ┃   ┃
┣━━━╋━━━╋   ╋━━━╋   ╋   ╋   ╋   ╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋   ┫
┃       ┃           ┃   ┃   ┃       ┃       ┃   ┃       ┃   ┃
┣   ╋   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋   ╋━━━╋━━━╋   ╋   ╋   ╋   ┫
┃   ┃       ┃   ┃       ┃       ┃   ┃           ┃   ┃       ┃
┣   ╋━━━╋   ╋   ╋   ╋━━━╋   ╋   ╋   ╋   ╋━━━╋━━━╋   ╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃           ┃   ┃       ┃       ┃           ┃
┣   ╋   ╋   ╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋   ┫
┃   ┃                   ┃       ┃           ┃           ┃   ┃
┣   ╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋   ╋   ┫
┃   ┃   ┃       ┃           ┃       ┃   ┃               ┃   ┃
┣   ╋━━━

In [11]:
print(Maze.gen_fusion(15, 15))

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃           ┃               ┃       ┃   ┃   ┃       ┃   ┃   ┃
┣━━━╋   ╋━━━╋   ╋━━━╋   ╋   ╋   ╋━━━╋   ╋   ╋━━━╋   ╋   ╋   ┫
┃       ┃       ┃       ┃           ┃       ┃   ┃       ┃   ┃
┣   ╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋   ╋   ╋━━━╋   ╋   ┫
┃   ┃                   ┃   ┃   ┃   ┃       ┃               ┃
┣━━━╋   ╋━━━╋━━━╋   ╋   ╋   ╋   ╋━━━╋━━━╋   ╋   ╋━━━╋   ╋   ┫
┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃           ┃   ┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋   ╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋   ╋━━━╋━━━╋   ┫
┃       ┃       ┃   ┃   ┃   ┃       ┃               ┃       ┃
┣━━━╋   ╋━━━╋   ╋━━━╋   ╋   ╋   ╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋━━━┫
┃       ┃                       ┃       ┃   ┃           ┃   ┃
┣━━━╋   ╋   ╋━━━╋━━━╋   ╋   ╋━━━╋   ╋━━━╋   ╋   ╋   ╋━━━╋   ┫
┃   ┃   ┃   ┃           ┃               ┃   ┃   ┃           ┃
┣   ╋   ╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋   ╋   ╋━━━╋   ╋   ┫
┃                   ┃   ┃   ┃   ┃           ┃   ┃   ┃   ┃   ┃
┣━━━╋   

In [12]:
laby = Maze.gen_wilson(12, 12)
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃           ┃                   ┃       ┃       ┃
┣━━━╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋   ╋   ╋━━━┫
┃           ┃   ┃   ┃   ┃   ┃   ┃   ┃           ┃
┣━━━╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋━━━╋   ╋━━━╋━━━┫
┃       ┃                   ┃                   ┃
┣━━━╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋   ╋   ┫
┃   ┃           ┃   ┃       ┃           ┃   ┃   ┃
┣   ╋   ╋━━━╋━━━╋   ╋━━━╋   ╋   ╋   ╋   ╋   ╋   ┫
┃       ┃   ┃       ┃           ┃   ┃   ┃   ┃   ┃
┣━━━╋   ╋   ╋   ╋   ╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋   ┫
┃   ┃   ┃       ┃       ┃   ┃       ┃   ┃   ┃   ┃
┣   ╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋   ╋━━━╋   ╋   ╋━━━┫
┃       ┃   ┃   ┃           ┃                   ┃
┣   ╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋   ╋   ╋   ╋━━━╋   ┫
┃           ┃       ┃           ┃   ┃   ┃       ┃
┣   ╋━━━╋   ╋   ╋━━━╋   ╋   ╋━━━╋━━━╋   ╋━━━╋━━━┫
┃       ┃           ┃   ┃   ┃   ┃               ┃
┣━━━╋━━━╋   ╋━━━╋━━━╋   ╋   ╋   ╋━━━╋━━━╋   ╋   ┫
┃   ┃   ┃   ┃   ┃       ┃       ┃           ┃   ┃


In [79]:
laby = Maze.gen_fusion(15, 15)
labyTest = laby
solution = laby.solve_dfs((0, 0), (14, 14))
str_solution = {c:'*' for c in solution}
str_solution[( 0,  0)] = 'D'
str_solution[(14, 14)] = 'A'
print(laby.overlay(str_solution))

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃ D ┃       ┃       ┃               ┃                   ┃   ┃
┣   ╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋   ┫
┃ *   *   *   * ┃               ┃   ┃       ┃       ┃       ┃
┣   ╋   ╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋   ╋   ╋   ╋   ┫
┃   ┃       ┃ * ┃   ┃   ┃   ┃   ┃   ┃       ┃   ┃       ┃   ┃
┣━━━╋   ╋━━━╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋━━━╋━━━╋━━━╋   ╋━━━┫
┃   ┃   ┃   ┃ *   *   * ┃       ┃           ┃               ┃
┣   ╋━━━╋   ╋━━━╋━━━╋   ╋   ╋   ╋   ╋━━━╋━━━╋   ╋━━━╋   ╋   ┫
┃               ┃     * ┃   ┃           ┃   ┃   ┃       ┃   ┃
┣   ╋━━━╋━━━╋━━━╋   ╋   ╋   ╋━━━╋   ╋━━━╋   ╋   ╋━━━╋   ╋━━━┫
┃       ┃   ┃   ┃   ┃ * ┃   ┃   ┃           ┃   ┃   ┃   ┃   ┃
┣   ╋   ╋   ╋   ╋━━━╋   ╋━━━╋   ╋   ╋━━━╋━━━╋   ╋   ╋   ╋   ┫
┃   ┃   ┃   ┃       ┃ * ┃   ┃           ┃           ┃   ┃   ┃
┣━━━╋   ╋   ╋   ╋━━━╋   ╋   ╋   ╋━━━╋   ╋━━━╋━━━╋   ╋   ╋   ┫
┃   ┃       ┃   ┃   ┃ *         ┃           ┃       ┃       ┃
┣   ╋━━━

In [184]:
laby = Maze.gen_exploration(15, 15)
solution = laby.solve_dfs((0, 0), (14, 14))
try:
    str_solution = {c:'*' for c in solution}
except TypeError:
    print(laby)
str_solution[( 0,  0)] = 'D'
str_solution[(14, 14)] = 'A'
print(laby.overlay(str_solution))

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃                           ┃           ┃       ┃           ┃
┣━━━╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋   ╋   ╋━━━╋   ╋   ╋━━━╋   ┫
┃       ┃               ┃           ┃       ┃   ┃   ┃       ┃
┣   ╋   ╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋   ╋   ╋━━━┫
┃   ┃       ┃                           ┃   ┃   ┃   ┃       ┃
┣   ╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋   ╋   ╋━━━╋   ┫
┃   ┃           ┃   ┃       ┃       ┃       ┃   ┃   ┃       ┃
┣   ╋━━━╋━━━╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋━━━╋   ╋   ╋━━━╋━━━┫
┃   ┃       ┃       ┃   ┃       ┃   ┃           ┃           ┃
┣   ╋   ╋   ╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋   ┫
┃   ┃   ┃       ┃   ┃               ┃                   ┃   ┃
┣   ╋   ╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋━━━╋   ┫
┃   ┃   ┃       ┃       ┃       ┃       ┃                   ┃
┣   ╋   ╋   ╋━━━╋   ╋   ╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃           ┃   ┃       ┃       ┃   ┃               ┃
┣   ╋   

In [11]:
laby = Maze.gen_wilson(15, 15)
solution = laby.solve_rhr((0, 0), (14, 14))
str_solution = {c:'*' for c in solution}
str_solution[( 0,  0)] = 'D'
str_solution[(14, 14)] = 'A'
print(laby.overlay(str_solution))

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃ D ┃ *   *   *     ┃   ┃           ┃                   ┃   ┃
┣   ╋   ╋   ╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋   ╋━━━╋   ╋━━━╋   ┫
┃ *   * ┃ * ┃ *   *   *   * ┃       ┃   ┃       ┃           ┃
┣   ╋   ╋   ╋━━━╋━━━╋   ╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋   ┫
┃ * ┃ * ┃ *   *   * ┃ * ┃ *   *   * ┃   ┃   ┃           ┃   ┃
┣   ╋━━━╋━━━╋   ╋   ╋   ╋━━━╋   ╋   ╋   ╋   ╋   ╋   ╋━━━╋   ┫
┃ *   *   * ┃ * ┃ * ┃ * ┃ *   * ┃ * ┃   ┃   ┃   ┃   ┃       ┃
┣   ╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋   ╋   ╋   ╋━━━╋   ╋   ┫
┃ *   * ┃ *   * ┃ * ┃ *   *   *   *         ┃   ┃   ┃   ┃   ┃
┣━━━╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋   ╋━━━╋   ╋   ╋━━━╋   ┫
┃ *   *   * ┃ *   *   *   *   * ┃ * ┃           ┃   ┃   ┃   ┃
┣━━━╋   ╋━━━╋   ╋   ╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━┫
┃ * ┃ * ┃ * ┃ * ┃ * ┃ *   * ┃ *   *   *   *   * ┃           ┃
┣   ╋   ╋   ╋   ╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋   ╋━━━┫
┃ * ┃ *   * ┃ * ┃ *   * ┃ * ┃ *   *   *   *   *     ┃       ┃
┣   ╋   

In [12]:
laby.distance_geo((0, 0), (15, 25))

TypeError: object of type 'NoneType' has no len()